# 07 – Deployment with Streamlit

This notebook demonstrates how to deploy the best model using a simple Streamlit app.  The app allows a user to enter input features and receive a predicted CTR.


In [ ]:

import streamlit as st
import joblib
import pandas as pd
import os

# Load the best model (choose based on evaluation results)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
models_dir = os.path.join(project_root, 'models')
model_path = os.path.join(models_dir, 'Stacking_model.pkl')  # adjust if another model is best
model = joblib.load(model_path)

st.title('CTR Prediction App')
st.write('Enter feature values to get a predicted click-through rate.')

# Example fields – these should match the dataset features
user_id = st.number_input('User ID', min_value=0, step=1)
adgroup_id = st.number_input('Ad group ID', min_value=0, step=1)
# Add additional feature inputs as needed based on the model

# When the user clicks predict
if st.button('Predict'):
    input_df = pd.DataFrame({
        'user': [user_id],
        'adgroup_id': [adgroup_id],
        # Add other features here with appropriate names and types
    })
    proba = model.predict_proba(input_df)[0, 1]
    st.write(f'Predicted click probability: {proba:.4f}')
